In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import trimesh

scene = trimesh.Scene()

In [ ]:
import my_code.diffusion_training.data_loading as data_loading

train_dataset = data_loading.get_val_dataset(
    'FAUST_orig', 'train', 200, canonicalize_fmap=None
    )[1]
test_dataset = data_loading.get_val_dataset(
    'FAUST_orig', 'test', 200, canonicalize_fmap=None
)[1]

In [ ]:
import my_code.utils.plotting_utils as plotting_utils

l = 0
h = 32

fig, axs = plt.subplots(1, 3, figsize=(10, 5))

plotting_utils.plot_Cxy(fig, axs[0], train_dataset[55]['second']['C_gt_xy'][0],
                        'C_xy 55', l, h, show_grid=False, show_colorbar=False)
plotting_utils.plot_Cxy(fig, axs[1], train_dataset[67]['second']['C_gt_xy'][0],
                        'C_xy 67', l, h, show_grid=False, show_colorbar=False)
plotting_utils.plot_Cxy(fig, axs[2], train_dataset[78]['second']['C_gt_xy'][0],
                        'C_xy 78', l, h, show_grid=False, show_colorbar=False)



plt.show()

## Gaussian noise

In [ ]:
import utils.geometry_util as geometry_util

verts = train_dataset[15]['second']['verts']
faces = train_dataset[15]['second']['faces']

# augment the vertices
verts_aug = geometry_util.data_augmentation(verts.unsqueeze(0),
                                        # rot_x=0.0, rot_y=90.0, rot_z=0.0,
                                        rot_x=0.0, rot_y=0.0, rot_z=0.0,
                                        std=0.01,
                                        scale_min=1, scale_max=1
                                        )[0]

In [ ]:
scene.geometry.clear()

scene.add_geometry(trimesh.Trimesh(
    vertices=verts + torch.tensor([0, 0, 0]),
    faces=faces)
)

scene.add_geometry(trimesh.Trimesh(
    vertices=verts_aug + torch.tensor([1, 0, 0]),
    faces=faces)
)
    
    
axis = trimesh.creation.axis(axis_length=1)
scene.add_geometry(axis)
scene.show()

## Noise along normal

In [ ]:
import utils.geometry_util as geometry_util

verts = train_dataset[15]['second']['verts']
faces = train_dataset[15]['second']['faces']


vert_normals = geometry_util.vertex_normal(verts, faces)

In [ ]:
# for each vertex, add gaussian noise along the normal
std = 0.01
noise_clip = 0.05

# random noise
noise = std * torch.randn(verts.shape[0]).to(verts.device)
noise = noise.clamp(-noise_clip, noise_clip)
# noise = noise.repeat(3, 1).t()
noise = noise.unsqueeze(1)

verts_norm_noise = verts + noise * vert_normals

# assert torch.allclose(verts + noise.unsqueeze(1) * vert_normals, verts + noise.repeat(3, 1).t() * vert_normals)

In [ ]:
noise

In [ ]:
scene.geometry.clear()

scene.add_geometry(trimesh.Trimesh(
    vertices=verts + torch.tensor([0, 0, 0]),
    faces=faces)
)
scene.add_geometry(trimesh.Trimesh(
    vertices=verts_aug + torch.tensor([1, 0, 0]),
    faces=faces)
)
scene.add_geometry(trimesh.Trimesh(
    vertices=verts_norm_noise + torch.tensor([2, 0, 0]),
    faces=faces)
)
    
    
axis = trimesh.creation.axis(axis_length=1)
scene.add_geometry(axis)
scene.show()

In [ ]:
import importlib
importlib.reload(geometry_util)

In [ ]:
import utils.geometry_util as geometry_util

verts = train_dataset[15]['second']['verts']
faces = train_dataset[15]['second']['faces']

In [ ]:
verts_gauss = geometry_util.data_augmentation(verts.unsqueeze(0),
                                              faces=faces.unsqueeze(0),
                                              rot_x=0,
                                              rot_y=90.0, 
                                              rot_z=0, 
                                              std=0.01,
                                              noise_clip=0.05,
                                              along_normal=False, 
                                              scale_min=0.9, 
                                              scale_max=1.1
)
verts_normal = geometry_util.data_augmentation(verts.unsqueeze(0),
                                              faces=faces.unsqueeze(0),
                                              rot_x=0,
                                              rot_y=90.0, 
                                              rot_z=0, 
                                              std=0.01,
                                              noise_clip=0.05,
                                              along_normal=True, 
                                              scale_min=0.9, 
                                              scale_max=1.1
)


scene.geometry.clear()

scene.add_geometry(trimesh.Trimesh(
    vertices=verts + torch.tensor([0, 0, 0]),
    faces=faces)
)
scene.add_geometry(trimesh.Trimesh(
    vertices=verts_gauss[0] + torch.tensor([1, 0, 0]),
    faces=faces)
)
scene.add_geometry(trimesh.Trimesh(
    vertices=verts_normal[0] + torch.tensor([2, 0, 0]),
    faces=faces)
)
    
axis = trimesh.creation.axis(axis_length=1)
scene.add_geometry(axis)
scene.show()

## Check the generated meshes

In [ ]:
import my_code.sign_canonicalization.training as sign_training

train_folder = 'FAUST_orig_train_rot_180_180_180_normal_True_noise_0.01_-0.05_0.05_lapl_mesh_scale_0.9_1.1'
test_folder = 'FAUST_orig_test_rot_180_180_180_normal_True_noise_0.01_-0.05_0.05_lapl_mesh_scale_0.9_1.1'

train_shapes, train_diff_folder = sign_training.load_cached_shapes(
    f'/home/s94zalek_hpc/shape_matching/data_sign_training/train/{train_folder}',
    lapl_type='mesh'
)

test_shapes, test_diff_folder = sign_training.load_cached_shapes(
    f'/home/s94zalek_hpc/shape_matching/data_sign_training/test/{test_folder}',
    lapl_type='mesh'
)

In [ ]:
# add 4 random training shapes to trimesh scene

# np.random.shuffle(train_shapes)
scene.geometry.clear()

rand_idx_train = np.random.randint(0, len(train_shapes), 5)
rand_idx_test = np.random.randint(0, len(test_shapes), 5)

for i, idx in enumerate(rand_idx_train):
    scene.add_geometry(trimesh.Trimesh(
        vertices=train_shapes[idx]['verts'] + torch.tensor([i, 0, 0]),
        faces=train_shapes[idx]['faces']))
    
for i, idx in enumerate(rand_idx_test):
    scene.add_geometry(trimesh.Trimesh(
        vertices=test_shapes[idx]['verts'] + torch.tensor([i, -1, 0]),
        faces=test_shapes[idx]['faces']))
    
axis = trimesh.creation.axis(axis_length=1)
scene.add_geometry(axis)
scene.show()